In [ ]:
import os
import cv2
import pandas as pd
import torch
import time
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import torchvision.models as models
from torchvision.utils import make_grid
from torch.utils.data import RandomSampler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
batch_size = 64
target_size = (224*224*3)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
])

train_dir = "/kaggle/input/100-bird-species/train/"
test_dir = "/kaggle/input/100-bird-species/test/"
val_dir = "/kaggle/input/100-bird-species/valid/"

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    total_loss = 0

    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    return total_loss / size

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size

    return test_loss, 100 * correct

In [ ]:
def train_and_evaluate_model(model, train_dataloader, test_dataloader, loss_fn, optimizer, n_epochs):
    losses_train = []
    losses_test = []
    accuracies_test = []

    for t in range(n_epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        loss_train = train(train_dataloader, model, loss_fn, optimizer)
        loss_test, accuracy_test = test(test_dataloader, model, loss_fn)
        print("Loss train:", loss_train, ", loss test:", loss_test, ", accuracy test:", accuracy_test)

        losses_train.append(loss_train)
        losses_test.append(loss_test)
        accuracies_test.append(accuracy_test)

    print("Done!")
    return model, losses_train, losses_test, accuracies_test

In [ ]:
start_time = time.time()

# Define the ResNet model
resnet = models.resnet101(weights=True)
resnet.fc = nn.Linear(resnet.fc.in_features, 525)  # Adjust for number of classes in the Dataset

# Freeze all layers
for param in resnet.parameters():
    param.requires_grad = False

# Unfreeze the fully connected layer
for param in resnet.fc.parameters():
    param.requires_grad = True

# Define transformations
resnet_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load data using ImageFolder
train_data_resnet = ImageFolder(root=train_dir, transform=resnet_transform)
valid_data_resnet = ImageFolder(root=val_dir, transform=resnet_transform)

# Create DataLoader instances
train_dataloader = DataLoader(train_data_resnet, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_data_resnet, batch_size=32, shuffle=False)

# Define loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.fc.parameters(), lr=1e-3)  # Optimize only the last layer

# Move model to the appropriate device
resnet.to(device)

# Call the training and evaluation function
resnet, resnet_losses_train, resnet_losses_test, resnet_accuracies_test = train_and_evaluate_model(
    model=resnet,
    train_dataloader=train_dataloader,
    test_dataloader=valid_dataloader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    n_epochs=10
)

end_time = time.time()
execution_time = end_time - start_time
print(f'Время выполнения: {execution_time} секунд')

# Plotting
plt.plot(resnet_losses_train, label="Train loss")
plt.plot(resnet_losses_test, label="Test loss")
plt.xlabel("Epoch")
plt.legend()
plt.show()

In [ ]:
# Save model's state_dict
torch.save(resnet.state_dict(), 'model_weights.pth')

In [ ]:
plt.plot(resnet_accuracies_test, label="Test loss")
plt.xlabel("Epoch")
plt.legend()
plt.show()